In [16]:
# !pip install pandas

In [17]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [ ]:
url = 'https://www.tuhocielts.vn/tai-hon-3000-tu-tieng-anh-thong-dung-theo-chu-de-pdf-free/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}


response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [19]:
content_div = soup.find('div', class_='entry-content')
text_vocab = []
current_title = ""

In [20]:

for elem in content_div.find_all(['h3', 'p']):
    if elem.name == 'h3':
        raw_title = elem.get_text(strip=True)
        # Lấy phần sau từ "chủ đề" (không phân biệt hoa thường)
        match_title = re.search(r"(chủ đề.*)", raw_title, re.IGNORECASE)
        current_title = match_title.group(1).strip() if match_title else raw_title
    elif elem.name == 'p':
        lines = elem.get_text().split('\n')
        for line in lines:
            line = line.strip()
            if not line or ':' not in line:
                continue

            match = re.match(r"""^
                ([\w\-\s]+)               # Từ tiếng Anh
                (?:\s*/([^/]+)/)?        # Phiên âm (tùy chọn)
                (?:\s*\(([^)]+)\))?      # Loại từ (tùy chọn)
                \s*:\s*(.+)$             # Nghĩa tiếng Việt
            """, line, re.VERBOSE)

            if match:
                word = match.group(1).strip()
                ipa = f"/{match.group(2).strip()}/" if match.group(2) else ""
                word_type = match.group(3).strip() if match.group(3) else ""
                meaning = match.group(4).strip()
                
                text_vocab.append({
                    "Topic": current_title,
                    "English": word,
                    "IPA": ipa,
                    "Type": word_type,
                    "Vietnamese": meaning
                })


In [ ]:
# Ghi ra file JSON
with open("vocabulary.json", "w", encoding="utf-8") as f:
    json.dump(text_vocab, f, ensure_ascii=False, indent=4)